In [1]:

import glob, sys, sqlite3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from colorama import Fore, Back, Style
from matplotlib.ticker import PercentFormatter
from matplotlib.pyplot import figure

# Noms de colonnes à utiliser
names = ['Code Domaine', 'Domaine', 'Code Pays', 'Pays', 'Code Élément', 'Élément', 'Code Produit',
         'Produit', 'Code Année', 'Année', 'Unité', 'Valeur', 'Symbole', 'Description du Symbole']

fns = glob.glob('*/*.csv')
conn = sqlite3.connect('bdd.sqlite')

# Dictionnaires des csv
dfs = {}

for fn in fns:

    fns = fn.split('/')[-1][13:-4]

    if fns == 'Groupe_de_pays':
        df = pd.read_csv(fn)
    else:
        df = pd.read_csv(fn, names=names, header=0)

    print(f" - {fn:50s} ({df.shape[0]:6d}, {df.shape[1]:2d}) - {Fore.BLACK}{Back.GREEN}{fns}{Style.RESET_ALL}")
    
    # Récupération des données dans un sql
    df.to_sql(fns, con=conn, if_exists='replace')
    # Et dans le dictionnaire
    dfs[fns] = df

plt.rcParams["figure.figsize"] = (22,10)


 - data/FAOSTAT_2013_vegetal.csv                      (104871, 14) - vegetal
 - data/FAOSTAT_2013_animal.csv                       ( 37166, 14) - animal
 - data/FAOSTAT_2013_superficie.csv                   (   238, 14) - superficie
 - data/FAOSTAT_2023_Groupe_de_pays.csv               (  1941,  7) - Groupe_de_pays
 - data/FAOSTAT_2013_population.csv                   (   175, 14) - population
 - data/FAOSTAT_2013_sous_alimentation.csv            (   204, 14) - sous_alimentation
 - data/FAOSTAT_2013_cereal.csv                       ( 16057, 14) - cereal


# Question 1

In [2]:

# Récupèration des noms de pays et de la population
df_pop = pd.DataFrame(dfs['population'])[['Code Pays', 'Pays', 'Valeur']]

# On fait la somme
print(df_pop['Valeur'].sum() * 1000)
# Le résultat est erroné car la Chine est représenté plusieurs fois

# On sélectionne tout les doublons de la Chine commençant par "China, "
df_china = df_pop[df_pop['Pays'].str.startswith('China,')]

#print(df_china)

# On les drop
df_pop.drop(df_china.index, inplace=True)

nbr_total_humain = df_pop['Valeur'].sum() * 1000
print(nbr_total_humain)


# En prévention des autres questions on va supprimer les doublons aussi dans tout les DataFrames
for dfn in dfs:
    for code in df_china['Code Pays']:
        
        dfs[dfn].drop(dfs[dfn][dfs[dfn]['Code Pays'] == code].index, inplace=True)


8413993000
6997326000


# Question 2

In [3]:

# prod + imp - exp + var = disp = ali + sem + per + trai + aut + nou

df = dfs['vegetal']

# On récupère pour l'exemple du Blé en France
df = df.query("Pays == 'France' and Produit == 'Blé'")[['Élément', 'Valeur']].set_index('Élément')

prod = df.at['Production', 'Valeur']
imp  = df.at['Importations - Quantité', 'Valeur']
exp  = df.at['Exportations - Quantité', 'Valeur']
var  = df.at['Variation de stock', 'Valeur']
disp = df.at['Disponibilité intérieure', 'Valeur']
ali  = df.at['Aliments pour animaux', 'Valeur']
sem  = df.at['Semences', 'Valeur']
per  = df.at['Pertes', 'Valeur']
trai = df.at['Traitement', 'Valeur']
aut  = df.at['Autres Utilisations', 'Valeur']
nou  = df.at['Nourriture', 'Valeur']

print("prod + imp - exp + var             =", prod + imp - exp + var)
print("disp                               =", disp)
print("ali + sem + per + trai + aut + nou =", ali + sem + per + trai + aut + nou)

prod + imp - exp + var             = 20298.0
disp                               = 20298.0
ali + sem + per + trai + aut + nou = 20298.0


# Question 3

In [4]:

# On récupère les csv
animaux = pd.DataFrame(dfs['animal'])
vegetaux = pd.DataFrame(dfs['vegetal'])

# Ajout d'une colonne 'Origine' dans animaux et vegetaux
animaux['Origine'] = 'Animale'
vegetaux['Origine'] = 'Végétale'

# Union de Végétaux et Animaux
aliments = pd.concat([vegetaux, animaux])

# Projection
aliments = aliments[[
    'Pays',
    'Code Pays',
    'Produit',
    'Origine',
    'Élément',
    'Valeur'
]]

# On prend uniquement les éléments qui nous intéresse
kcal = "Disponibilité alimentaire (Kcal/personne/jour)"
prot = "Disponibilité de protéines en quantité (g/personne/jour)"
poid = "Disponibilité alimentaire en quantité (kg/personne/an)"
aliments.query("Élément == @kcal or Élément == @prot or Élément == @poid", inplace=True)

# Table pivot sur aliments
aliments = aliments.pivot_table('Valeur', index=['Code Pays','Pays','Produit','Origine'], columns='Élément')

# Reset index
aliments.reset_index(inplace=True)

# Joindre population et aliments
aliments = pd.merge(aliments, dfs['population'][['Pays', 'Valeur']])

# Renommer les colonnes
aliments.rename(columns={
    'Disponibilité alimentaire (Kcal/personne/jour)'           : 'kcal/pers/an',
    'Disponibilité de protéines en quantité (g/personne/jour)' : 'kg prot/pers/an',
    'Disponibilité alimentaire en quantité (kg/personne/an)'   : 'kg/pers/an',
    'Valeur' : 'population'
}, inplace=True)

# On fait les calculs pour correspondre aux unités
aliments['population'] *= 1000
aliments['kcal/pers/an'] *= 365
aliments['kg prot/pers/an'] *= 365
aliments['kg prot/pers/an'] /= 1000

aliments['énergie/poids'] = aliments['kcal/pers/an'] / aliments['kg/pers/an']
aliments['kg prot/poids'] = aliments['kg prot/pers/an'] / aliments['kg/pers/an']

# Remplacer les Nan et Inf
aliments.replace(np.nan, 0, inplace=True)
aliments.replace(np.inf, 0, inplace=True)

# Affichage
print('Résultat : ', aliments.shape[1], 'colonnes, ', aliments.shape[0], 'lignes')
aliments[['Pays', 'Produit', 'Origine', 'kcal/pers/an', 'kg prot/pers/an', 'population']].sample(10)


Résultat :  10 colonnes,  4405 lignes


,Pays,Produit,Origine,kcal/pers/an,kg prot/pers/an,population
1888,Côte d'Ivoire,Animaux Aquatiques Autre,Animale,0.0,0.00000,20316000
200,Bahamas,"Racines, Nda",Végétale,365.0,0.00365,377000
2188,Lesotho,Sorgho,Végétale,28470.0,0.88695,2074000
3707,Eswatini,"Céréales, Autres",Végétale,365.0,0.01460,1250000
2224,Madagascar,Graines Colza/Moutarde,Végétale,0.0,0.00000,22925000
3040,Panama,Viande de Suides,Animale,17155.0,1.31035,3864000
2686,Vanuatu,Riz (Eq Blanchi),Végétale,150745.0,2.66085,253000
2431,Maldives,Plantes Aquatiques,Animale,0.0,0.00000,345000
586,Canada,Huile de Germe de Maïs,Végétale,13870.0,0.00000,35182000
2602,Mozambique,"Poissons Marins, Autres",Animale,2920.0,0.49275,25834000


# Question 4

In [5]:

print("Moyenne du rapport énergie/poids des oeufs (kcal/kg) :")

print(np.mean(aliments.query("Produit == 'Oeufs'")['énergie/poids']))

print()

aliments[['Pays', 'Produit', 'Origine', 'énergie/poids']].sample(10)


Moyenne du rapport énergie/poids des oeufs (kcal/kg) :
1362.3173838894493



,Pays,Produit,Origine,énergie/poids
1452,Ghana,Oignons,Végétale,379.643388
3716,Eswatini,Huil Plantes Oleif Autr,Végétale,7300.000000
428,Belize,Huile de Tournesol,Végétale,8910.798122
1691,Guyana,"Mollusques, Autres",Animale,0.000000
2003,Kazakhstan,Graisses Animales Crue,Animale,7300.000000
3614,Suriname,Arachides Decortiquees,Végétale,5840.000000
1511,Kiribati,Huil Plantes Oleif Autr,Végétale,8760.000000
666,Cabo Verde,Haricots,Végétale,3415.944541
1654,Guyana,Boissons Alcooliques,Végétale,2936.781609
1192,France,"Légumes, Autres",Végétale,282.101695


# Question 5 part 1

In [6]:

# On calcul la moyenne de l'énergie sur le poids pour chaque produit, puis on trie dans l'ordre décroissant
prod_en_poids = aliments[['Produit', 'énergie/poids']].groupby('Produit').mean().sort_values('énergie/poids', ascending=False)

prod_en_poids.head(20)


,énergie/poids
Produit,
Huile de Palme,8712.998046
Huile de Soja,8671.981316
Graisses Animales Crue,7714.200488
Huil Plantes Oleif Autr,7497.492849
Huile de Tournesol,7264.736416
"Beurre, Ghee",6607.422927
Huile de Son de Riz,6208.686869
Huile d'Olive,6134.059095
Huile de Coco,6066.931493


# Question 5 part 2

In [7]:

# Pareil mais pour la quantité de protéine
prod_prot_poids = aliments[['Produit', 'kg prot/poids']].groupby('Produit').mean().sort_values('kg prot/poids', ascending=False)

prod_prot_poids.head(20)


,kg prot/poids
Produit,
Arachides Decortiquees,0.245061
Soja,0.230789
Haricots,0.202210
Légumineuses Autres,0.197367
Pois,0.192526
Abats Comestible,0.176632
"Viande, Autre",0.160918
Viande de Bovins,0.146130
Viande d'Ovins/Caprins,0.137785


# Question 6

In [8]:

df = dfs['vegetal'][['Élément', 'Produit', 'Valeur']]

# On fait la somme de la disponibilité intérieur de tous les pays pour chaque produit
df = df.query("Élément == 'Disponibilité intérieure'").groupby('Produit').sum()

# On merge les infos trouvés plus haut
df = pd.merge(df, prod_en_poids, left_index = True, right_index = True)

df.rename(columns={'Valeur' : 'poids (kg)', 'énergie/poids' : 'kcal/kg'}, inplace=True)

# On fait les calculs pour correspondre aux unités
df['poids (kg)'] *= 1000000

# On ajoute la colonne énergie total
df['énergie (kcal)'] = df['poids (kg)'] * df['kcal/kg']

df.head(5)


,poids (kg),kcal/kg,énergie (kcal)
Produit,,,
"Agrumes, Autres",1.217700e+10,73.585330,8.960486e+11
Aliments pour enfants,8.600000e+07,2619.335821,2.252629e+11
Ananas,2.328200e+10,301.483372,7.019136e+12
Arachides Decortiquees,2.969400e+10,5502.020083,1.633770e+14
Avoine,2.340700e+10,1319.777192,3.089202e+13


# Question 7

In [9]:

# La somme de l'énergie de chaque produit
somme_k = df['énergie (kcal)'].sum()

# La consommation moyenne de calorie d'un humain par an
conso_h_k = 2400 * 365

# On refais la même chose que l'exercice précédent mais pour la protéine
df = pd.merge(df, prod_prot_poids, left_index=True, right_index=True)

df['protéine (kg)'] = df['poids (kg)'] * df['kg prot/poids']

# La quantité total de protéine
somme_p = df['protéine (kg)'].sum()

# La consommation moyenne de protéine d'un humain par an
conso_h_p = 0.048 * 365

print("Calorie (kcal)")
print("  Energie total :", somme_k, "kcal")
print("  Energie consommé par un humain en un an :", conso_h_k, "kcal")
print("  Nombre d'humain nourris en calorie :", somme_k / conso_h_k)
print("  Soit", np.round((somme_k / conso_h_k) / nbr_total_humain * 100, 2), "% de la population mondiale")
print()
print("Protéine (kg)")
print("  Protéine total :", somme_p, "kg")
print("  Protéine consommé par un humain en un an :", conso_h_p, "kg")
print("  Nombre d'humain nourris en protéine:", somme_p / conso_h_p)
print("  Soit", np.round((somme_p / conso_h_p) / nbr_total_humain * 100, 2), "% de la population mondiale")

Calorie (kcal)
  Energie total : 1.1745547706429438e+16 kcal
  Energie consommé par un humain en un an : 876000 kcal
  Nombre d'humain nourris en calorie : 13408159482.225386
  Soit 191.62 % de la population mondiale

Protéine (kg)
  Protéine total : 295597673820.1998 kg
  Protéine consommé par un humain en un an : 17.52 kg
  Nombre d'humain nourris en protéine: 16872013345.901817
  Soit 241.12 % de la population mondiale


# Question 8



In [10]:

df = dfs['vegetal'][['Élément', 'Produit', 'Valeur']]

# On récupère les bonnes données en faisant la somme mondiale
nou = 'Nourriture'
ali  = 'Aliments pour animaux'
per  = 'Pertes'
df = df.query("Élément == @nou or Élément == @ali or Élément == @per").groupby(['Élément', 'Produit']).sum().reset_index()

df = df.pivot_table('Valeur', 'Produit', 'Élément').replace(np.nan, 0)

df.rename(columns={nou : 'nou', ali : 'ali', per : 'per'}, inplace=True)

# On passe des milliers de tonnes au kg
df['nou'] *= 1000000
df['ali'] *= 1000000
df['per'] *= 1000000

# On somme les trois colonnes pour chaque produit
df = df.sum(1)
df.name = 'poids total'

# On merge les infos trouvés plus haut
df = pd.merge(df, prod_en_poids, left_index = True, right_index = True)
df = pd.merge(df, prod_prot_poids, left_index = True, right_index = True)

# On calcul les énergie totales et les protéines totales
df['énergie total'] = df['poids total'] * df['énergie/poids']
df['protéine total'] = df['poids total'] * df['kg prot/poids']

somme_k = df['énergie total'].sum()
somme_p = df['protéine total'].sum()

print("Calorie (kcal)")
print("  Energie total :", somme_k, "kcal")
print("  Energie consommé par un humain en un an :", conso_h_k, "kcal")
print("  Nombre d'humain nourris en calorie :", somme_k / conso_h_k)
print("  Soit", np.round((somme_k / conso_h_k) / nbr_total_humain * 100, 2), "% de la population mondiale")
print()
print("Protéine (kg)")
print("  Protéine total :", somme_p, "kg")
print("  Protéine consommé par un humain en un an :", conso_h_p, "kg")
print("  Nombre d'humain nourris en protéine:", somme_p / conso_h_p)
print("  Soit", np.round((somme_p / conso_h_p) / nbr_total_humain * 100, 2), "% de la population mondiale")

Calorie (kcal)
  Energie total : 8715601111363210.0 kcal
  Energie consommé par un humain en un an : 876000 kcal
  Nombre d'humain nourris en calorie : 9949316337.172615
  Soit 142.19 % de la population mondiale

Protéine (kg)
  Protéine total : 199644249472.8996 kg
  Protéine consommé par un humain en un an : 17.52 kg
  Nombre d'humain nourris en protéine: 11395219718.772808
  Soit 162.85 % de la population mondiale
